In [1]:
import psycopg2 as db
from elasticsearch import Elasticsearch
from elasticsearch import helpers
import pandas as pd
from pandas import json_normalize
from faker import Faker


In [2]:
#connection
conn_string = "dbname='test' host='localhost' user='postgres' password='postgres'"
conn = db.connect(conn_string)
cur = conn.cursor()
fake = Faker()

In [37]:
#Inserting 1000 users into Posrgres SQL
data=[]
i=2
for r in range(1000):
    data.append((fake.name(),i,fake.street_address(),fake.city(),fake.zipcode()))
    i+=1
fakerData = tuple(data)
query = "insert into users (name,id,street,city,zip) values(%s,%s,%s,%s,%s)"
#print(cur.mogrify(query,fakerData[1]))
#cur.executemany(query,fakerData)
#conn.commit()

In [38]:
#Extracting in DataFrame
query2 = "select * from users"
df= pd.read_sql(query2,conn)
#df.to_json(orient='records')

In [6]:
#Insertng single user into Elastic Search - NoSQL
#es.__version__

es = Elasticsearch()
doc = {
    "name":"sanket",
    "id":1,
    "street":"67 St Germain Street",
    "city":"Boston",
    "zip":"02115"
}

result = es.index(index="users",document=doc)
print(result)

/Users/Sanket/opt/anaconda3/lib/python3.9/site-packages/elasticsearch/connection/base.py:200: ElasticsearchWarning: Elasticsearch built-in security features are not enabled. Without authentication, your cluster could be accessible to anyone. See https://www.elastic.co/guide/en/elasticsearch/reference/7.16/security-minimal-setup.html to enable security.
  warnings.warn(message, category=ElasticsearchWarning)


{'_index': 'users', '_type': '_doc', '_id': 'tUZqAn4Bpxm48dT7ogPk', '_version': 1, 'result': 'created', '_shards': {'total': 2, 'successful': 1, 'failed': 0}, '_seq_no': 0, '_primary_term': 1}


In [7]:
#Inserting 1000 users into Elasticsearch

dataSource=[
    {
        "_index":"users",
        "_source":{
            "name":fake.name(),
            "id":x,
            "street":fake.street_address(),
            "city":fake.city(),
            "zip":fake.zipcode()
        }
    }
    for x in range(1000)
]
es = Elasticsearch()
result = helpers.bulk(es,dataSource)
print(result)

(1000, [])


In [5]:
#fetching all documents
doc = {
    "query":{
        "match_all":{}
    }
}
res = es.search(index="users",body=doc,size=10)
#print(res['hits']['hits'])
#Loading in DataFrame
df = json_normalize(res['hits']['hits'])
df

/var/folders/jl/0hp406v12rn91f04k5hcw9qm0000gn/T/ipykernel_1192/2016646758.py:7: DeprecationWarning: The 'body' parameter is deprecated for the 'search' API and will be removed in a future version. Instead use API parameters directly. See https://github.com/elastic/elasticsearch-py/issues/1698 for more information
  res = es.search(index="users",body=doc,size=10)


,_index,_type,_id,_score,_source.name,_source.id,_source.street,_source.city,_source.zip
0,users,doc,j4HS_n0BDjPf49Ryo3eO,1.0,sanket,1,67 St Germain Street,Boston,02115
1,users,doc,kYHT_n0BDjPf49RygXfS,1.0,sanket,1,67 St Germain Street,Boston,02115
2,users,doc,koHT_n0BDjPf49Ry2XfZ,1.0,sanket,1,67 St Germain Street,Boston,02115
3,users,doc,lIHV_n0BDjPf49RyrXdZ,1.0,sanket,1,67 St Germain Street,Boston,02115
4,users,doc,lYHZ_n0BDjPf49RyOneW,1.0,sanket,1,67 St Germain Street,Boston,02115
5,users,doc,loHd_n0BDjPf49RylHdC,1.0,Rachel Edwards,0,3589 Kimberly Coves,Lake Vincentfurt,84595
6,users,doc,l4Hd_n0BDjPf49RylHdC,1.0,Eric Cox,1,51828 Greene Underpass Suite 789,South Madison,17297
7,users,doc,mIHd_n0BDjPf49RylHdC,1.0,Nicholas Weber,2,8982 Gibson Wall,Lake Megantown,59749
8,users,doc,mYHd_n0BDjPf49RylHdC,1.0,Lisa Rogers,3,0946 Gabriel Tunnel,Karentown,84636
9,users,doc,moHd_n0BDjPf49RylHdC,1.0,Donald Jones,4,0668 Singh Plains Suite 701,Kathrynborough,72056


In [95]:
#Searching by query
doc = {
    "query":{
        "match":{
            "name":"sanket"
        }
    }
}
res = es.search(index="users",body=doc,size=10)
#print(res['hits']['hits'])
df = json_normalize(res['hits']['hits'])
df

/var/folders/jl/0hp406v12rn91f04k5hcw9qm0000gn/T/ipykernel_13534/732736616.py:9: DeprecationWarning: The 'body' parameter is deprecated for the 'search' API and will be removed in a future version. Instead use API parameters directly. See https://github.com/elastic/elasticsearch-py/issues/1698 for more information
  res = es.search(index="users",body=doc,size=10)
/Users/Sanket/opt/anaconda3/lib/python3.9/site-packages/elasticsearch/connection/base.py:200: ElasticsearchWarning: Elasticsearch built-in security features are not enabled. Without authentication, your cluster could be accessible to anyone. See https://www.elastic.co/guide/en/elasticsearch/reference/7.16/security-minimal-setup.html to enable security.
  warnings.warn(message, category=ElasticsearchWarning)


,_index,_type,_id,_score,_source.name,_source.id,_source.street,_source.city,_source.zip
0,users,doc,j4HS_n0BDjPf49Ryo3eO,8.645942,sanket,1,67 St Germain Street,Boston,02115
1,users,doc,kYHT_n0BDjPf49RygXfS,8.645942,sanket,1,67 St Germain Street,Boston,02115
2,users,doc,koHT_n0BDjPf49Ry2XfZ,8.645942,sanket,1,67 St Germain Street,Boston,02115
3,users,doc,lIHV_n0BDjPf49RyrXdZ,8.645942,sanket,1,67 St Germain Street,Boston,02115
4,users,doc,lYHZ_n0BDjPf49RyOneW,8.645942,sanket,1,67 St Germain Street,Boston,02115
5,users,doc,Z4He_n0BDjPf49Ry8X-o,8.645942,sanket,1,67 St Germain Street,Boston,02115
6,users,doc,IIHf_n0BDjPf49RyiosP,8.645942,sanket,1,67 St Germain Street,Boston,02115


In [96]:
#searching by Lucene sytax
res = es.search(index="users",q="name:sanket",size=10)
df = json_normalize(res['hits']['hits'])
df

/Users/Sanket/opt/anaconda3/lib/python3.9/site-packages/elasticsearch/connection/base.py:200: ElasticsearchWarning: Elasticsearch built-in security features are not enabled. Without authentication, your cluster could be accessible to anyone. See https://www.elastic.co/guide/en/elasticsearch/reference/7.16/security-minimal-setup.html to enable security.
  warnings.warn(message, category=ElasticsearchWarning)


,_index,_type,_id,_score,_source.name,_source.id,_source.street,_source.city,_source.zip
0,users,doc,j4HS_n0BDjPf49Ryo3eO,8.645942,sanket,1,67 St Germain Street,Boston,02115
1,users,doc,kYHT_n0BDjPf49RygXfS,8.645942,sanket,1,67 St Germain Street,Boston,02115
2,users,doc,koHT_n0BDjPf49Ry2XfZ,8.645942,sanket,1,67 St Germain Street,Boston,02115
3,users,doc,lIHV_n0BDjPf49RyrXdZ,8.645942,sanket,1,67 St Germain Street,Boston,02115
4,users,doc,lYHZ_n0BDjPf49RyOneW,8.645942,sanket,1,67 St Germain Street,Boston,02115
5,users,doc,Z4He_n0BDjPf49Ry8X-o,8.645942,sanket,1,67 St Germain Street,Boston,02115
6,users,doc,IIHf_n0BDjPf49RyiosP,8.645942,sanket,1,67 St Germain Street,Boston,02115
